# Imports

In [16]:
#export
import os
import sys
sys.path.append(os.path.join(os.getcwd(),'exp'))

import time
import json
import logging
import datetime
import numpy as np
from pathlib import Path
from shutil import copyfile
from collections import OrderedDict

import nb_dockertools as docker_tools
from nb_cycle import get_dist, dump_state, init_params, BaseConfigCycler, Param
from nb_bo import  dict_merge, Bo
from config import cfg

# Code 

In [2]:
#export
def run(new_state, inner_state, aux_cfg_files=None, name='cfg', gpus='0', **kwargs):
    logging.info(f'\n\tNew state for {name}:\n {json.dumps(new_state, indent=4)}\n')
    
    root = Path(cfg.DAG.RUNS)
    root = root.absolute()
    os.makedirs(root, exist_ok=True)
    
    run_path, configs_path = init_run(root)
    if aux_cfg_files is not None:
        for cfg_file in aux_cfg_files:
            file_name = os.path.basename(cfg_file)
            copyfile(cfg_file, configs_path/file_name)
    else:
        aux_cfg_files=[]
    run_cfg_file = dump_state(new_state, configs_path, name)
    if kwargs.get('hp_points',None):
        dump_state(kwargs['hp_points'], configs_path, 'hp', is_config=False, yaml_dump=False)
    
    docker_result = docker_tools.main(run_path, gpus=gpus)
    results = {'configs':[run_cfg_file] + aux_cfg_files}
    results['docker_results'] = {
        'metric':docker_result
    }
    results['state'] = inner_state 
    return results   

def init_run(root):
    timestamp = '{:%Y_%b_%d_%H_%M_%S_%f}'.format(datetime.datetime.now())
    run_dir = root/f'run_{timestamp}'
    conf_dir = run_dir/'configs'
    os.makedirs(conf_dir, exist_ok=True)
    return run_dir, conf_dir

In [6]:
#export
class Cycler(BaseConfigCycler):
    def init_map(self):
        return {
            'g':('generations', int, 150),
            'e':('exp_power', int, np.NaN),
            'f0':('dec_f0', int, np.NaN),
            'f1':('dec_f1', int, np.NaN),
            'f2':('dec_f2', int, np.NaN),
            'f3':('dec_f3', int, np.NaN),
            'mc':('mutate_chance', float, np.NaN),
            'cr':('crossover_chance', float, np.NaN),
            'co':('combine_chance', float, np.NaN)
        }

cycler = Cycler()
p9 = {'name':'co', 'sampling':'random', 'arr':get_dist(start=.01, end=.99, num=500, space='lin')}                           
p7 = {'name':'e', 'sampling':'sequential', 'arr':[1,5,10,15,25,50]}
p1 = {'name':'mc', 'sampling':'random', 'arr':get_dist(start=5e-4, end=5e-3, num=500, space='lin')}                         
p3 = {'name':'cr', 'sampling':'random', 'arr':get_dist(start=.01, end=.99,   num=500, space='lin')}

p2 = {'name':'f0', 'sampling':'random', 'arr':get_dist(start=2, end=20,   num=500, space='lin', to_int=True)}
p4 = {'name':'f1', 'sampling':'random', 'arr':get_dist(start=2, end=30,   num=500, space='lin', to_int=True)}
p6 = {'name':'f2', 'sampling':'random', 'arr':get_dist(start=2, end=30,   num=500, space='lin', to_int=True)}
p8 = {'name':'f3', 'sampling':'random', 'arr':get_dist(start=2, end=10,   num=500, space='lin', to_int=True)}


#params_dists = {p['name']:init_params([p]) for p in [p1,p3,p7,p9]}

def cycle_all(**kwargs):
    inner_state, new_state=cycler.create_state([p1,p3,p7,p9, p2,p4,p6,p8], kwargs['seq_id'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def cycle_exp(**kwargs):
    inner_state, new_state=cycler.create_state([p7], kwargs['seq_id'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def cycle_combine(**kwargs):
    inner_state, new_state=cycler.create_state([p9])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def cycle_mutate(**kwargs):
    inner_state, new_state=cycler.create_state([p1])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def cycle_crossover(**kwargs):
    inner_state, new_state=cycler.create_state([p3])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)


In [15]:
#export
n_parallel_processes = len(cfg.GPUS.IDS)
warm_list = ['/home/sokolov/work/cycler/dHPO/2020_May_21_18_34_23_hp.json',
            '/home/sokolov/work/cycler/dHPO/2020_May_21_19_39_08_hp.json',
            ]

params_static = {
    'generations':{'default':200, 'type':int},
}
params_genom = {
            'genom|mutate_chance':{'bounds':(0,.05), 'type':float, 'prior':'uniform', 'default':None},
            'genom|crossover_chance':{'bounds':(0,1), 'type':float, 'prior':'uniform', 'default':None},
            'genom|combine_chance':{'bounds':(0,1), 'type':float, 'prior':'uniform', 'default':None},
            }
params_post = {
    'post|exp_power':{'bounds':(1,15), 'type':int, 'prior':'uniform', 'default':None},
}

p_all = {}
[dict_merge(p_all, d) for d in [params_static, params_genom, params_post]]
b1 = Bo(n=n_parallel_processes, params=p_all, warm_list=warm_list)
b2 = Bo(n=n_parallel_processes, params=params_post, warm_list=warm_list)


def bo_all(**kwargs):
    inner_state, new_state=b1.create_state(points=kwargs['hp_points'], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def bo_exp(**kwargs):
    inner_state, new_state=b2.create_state(points=kwargs['hp_points'], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

# Tests

In [16]:
#export
if __name__ == '__main__':
    r1 = cycle_exp(seq_id=0, gpu=0)
    print(r1)
    r2 = cycle_mut(aux_cfg_files=r1['configs'], gpu=0)
    print(r2)
    print('all good!')

[PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_53_34_854649/output/data_2020_May_10_11:53:37/log.txt')]
{'configs': [PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_53_34_854649/configs/2020_May_10_14_53_34_cc_e.yaml')], 'docker_results': 0.06180047336965799}
[PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_53_57_423649/output/data_2020_May_10_11:54:00/log.txt')]
{'configs': [PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_53_57_423649/configs/2020_May_10_14_53_57_cc_m.yaml'), PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_53_34_854649/configs/2020_May_10_14_53_34_cc_e.yaml')], 'docker_results': 0.061617747880518436}
all good!


# Export

In [18]:
!python3 extra/n2s.py runner.ipynb

Converted runner.ipynb to exp/nb_runner.py


In [15]:
!python3 exp/nb_runner.py

[PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_27_53_473298/output/data_2020_May_10_11:27:56/log.txt')]
{'configs': [PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_27_53_473298/configs/2020_May_10_14_27_53_cc_e.yaml')], 'docker_results': 0.061637488193809986}
[PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_28_16_649084/output/data_2020_May_10_11:28:19/log.txt')]
{'configs': [PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_28_16_649084/configs/2020_May_10_14_28_16_cc_m.yaml'), PosixPath('/home/sokolov/work/cycler/crsch_cycle/cycler_runs/run_2020_May_10_14_27_53_473298/configs/2020_May_10_14_27_53_cc_e.yaml')], 'docker_results': 0.06321999430656433}
all good!
